In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle

import json
kaggle_token = {"username":"datastorm087","key":"f96ada8acdc2ce1a66d19c772cb2e9fc"}
with open("kaggle.json", "w") as f:
  token_json = json.dumps(kaggle_token)
  f.write(token_json)

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c data-storm-4

  0% 0.00/5.77M [00:00<?, ?B/s]
100% 5.77M/5.77M [00:00<00:00, 157MB/s]


In [4]:
! unzip data-storm-4.zip

Archive:  data-storm-4.zip
  inflating: Data-dictionary.xlsx    
  inflating: DataStorm4.0_KaggleProblemStatement.pdf  
  inflating: Historical-transaction-data.csv  
  inflating: Store-info.csv          
  inflating: Testing-data.csv        


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

store_info_data = pd.read_csv('Store-info.csv')
historical_data = pd.read_csv('Historical-transaction-data.csv')
testing_data = pd.read_csv('Testing-data.csv')

print(store_info_data.head())
print(historical_data.head())
print(testing_data.head())

   shop_id  shop_area_sq_ft shop_profile
0  SHOP047              528     Moderate
1  SHOP009              676         High
2  SHOP083              676          Low
3  SHOP117              676          Low
4  SHOP042              676          Low
        item_description          transaction_date  invoice_id customer_id  \
0     ORANGE BARLEY 1.5L  2021-12-11T00:00:00.000Z       147.0        BGXA   
1       GINGER BEER 1.5L  2021-10-17T00:00:00.000Z       371.0        IA25   
2        TONIC PET 500ML  2021-12-13T00:00:00.000Z       484.0        VN7V   
3          CREAM SODA 1L  2021-12-13T00:00:00.000Z       484.0        VN7V   
4  STRAWBERRY MILK 180ML  2021-10-23T00:00:00.000Z      1310.0        7S00   

   shop_id  item_price  quantity_sold  
0  SHOP008         220              2  
1  SHOP112         220              2  
2  SHOP008         160              2  
3  SHOP008         150              2  
4  SHOP112         210              5  
   shop_id  shop_profile
0  SHOP046          

In [6]:
historical_data['transaction_date'] = pd.to_datetime(historical_data['transaction_date']).dt.date

In [7]:
# Convert transaction_date column to datetime format
historical_data['transaction_date'] = pd.to_datetime(historical_data['transaction_date'])

# Group the data by shop_id and get the date range for each group
date_range = historical_data.groupby('shop_id')['transaction_date'].agg(['min', 'max'])

# Print the date range for each shop
print(date_range)

print()

# Group the data by date range for each shop and get the count of transactions
grouped_data = date_range.groupby(['min', 'max']).size().reset_index(name='count')

# Print the grouped data
print(grouped_data)

               min        max
shop_id                      
SHOP001 2021-10-15 2021-12-15
SHOP002 2021-10-15 2021-12-15
SHOP003 2021-10-15 2021-12-15
SHOP004 2021-10-15 2021-12-15
SHOP005 2021-10-15 2021-12-15
...            ...        ...
SHOP123 2021-10-15 2021-12-15
SHOP124 2021-10-15 2021-12-15
SHOP125 2021-10-15 2021-12-15
SHOP126 2021-10-15 2021-12-15
SHOP127 2021-10-15 2021-12-15

[124 rows x 2 columns]

         min        max  count
0 2021-10-15 2021-12-15    121
1 2021-10-16 2021-12-15      1
2 2021-10-22 2021-12-15      1
3 2021-12-10 2021-12-15      1


In [8]:
unique_items = historical_data['item_description'].unique()
for item in unique_items:
    print(item)


def remove_last_word(x):
    if isinstance(x, str):  # check if x is a string
        return ' '.join(x.split(' ')[:-1])
    else:
        return x

historical_data['item_description'] = historical_data['item_description'].apply(remove_last_word)

ORANGE BARLEY 1.5L
GINGER BEER 1.5L
TONIC PET 500ML
CREAM SODA 1L
STRAWBERRY MILK 180ML
CREAM SODA 1.5L
TWISTEE APPLE 1L
FIT O MIXED FRUIT 200ML
SODA PET 1.5L
FIT O MIXED FRUIT 1L
KIK COLA 1L
LEMONADE 500ML
FIT O ORANGE 1L
NECTO 1.5L
LEMONADE 1.5L
GINGER BEER 1L
FIT O MANGO 200ML
FIT O MANGO 1L
CREAM SODA 500ML
FIT O ORANGE 200ML
nan
CREAM SODA APPLE POP 1L
BOTTLED DRINKING WATER 500ML
SODA 500ML
GINGER BEER 500ML
SODA 1L
NECTO 500ML
ORANGE CRUSH 500ML
BOTTLED DRINKING WATER 1.5L
NECTO 1L
GINGER BEER SUGAR FREE 500ML
ORANGE CRUSH 1.5L
DRY GINGER ALE PET 500ML
LEMONADE 1L
ORANGE CRUSH 1L
GINGER BEER 400ML
CHOCOLATE MILK 180ML
LIME CRUSH JUICE 1L


**Create New Features**

*   Unique Customers per Store
*   Transactions per Store i.e. Unique Invoice Numbers per Store
*   Unique Items per Store
*   Items per Store
*   Revenue per Store

In [9]:
# Compute total number of unique customers
unique_customers = historical_data.groupby(['shop_id'])['customer_id'].nunique()

unique_customers = unique_customers.rename('unique_customers')

store_info_data = store_info_data.merge(unique_customers, on='shop_id')

In [10]:
# Compute total number of unique transactions
transactions = historical_data.groupby(['shop_id'])['invoice_id'].nunique()

transactions = transactions.rename('transactions')

store_info_data = store_info_data.merge(transactions, on='shop_id')

In [11]:
# Compute total number of unique items
unique_items = historical_data.groupby('shop_id')['item_description'].nunique()

unique_items = unique_items.rename('unique_items')

store_info_data = store_info_data.merge(unique_items, on='shop_id')

In [12]:
# Compute total number of items
items = historical_data.groupby(['shop_id'])['quantity_sold'].sum()

items = items.rename('items')

store_info_data = store_info_data.merge(items, on='shop_id')

In [13]:
# Compute total revenue
historical_data['revenue'] = historical_data['item_price'] * historical_data['quantity_sold']

revenue = historical_data.groupby('shop_id')['revenue'].sum()

revenue = revenue.rename('revenue')

store_info_data = store_info_data.merge(revenue, on='shop_id')

In [14]:
# Compute max item_price
max_item_price = historical_data.groupby('shop_id')['item_price'].max()

max_item_price = max_item_price.rename('max_item_price')

store_info_data = store_info_data.merge(max_item_price, on='shop_id')

In [15]:
# Compute min item_price
min_item_price = historical_data.groupby('shop_id')['item_price'].max()

min_item_price = min_item_price.rename('min_item_price')

store_info_data = store_info_data.merge(min_item_price, on='shop_id')

In [16]:
# Compute max quantity_sold
max_quantity_sold = historical_data.groupby('shop_id')['quantity_sold'].max()

max_quantity_sold = max_quantity_sold.rename('max_quantity_sold')

store_info_data = store_info_data.merge(max_quantity_sold, on='shop_id')

In [17]:
# Compute min quantity_sold
min_quantity_sold = historical_data.groupby('shop_id')['quantity_sold'].max()

min_quantity_sold = min_quantity_sold.rename('min_quantity_sold')

store_info_data = store_info_data.merge(min_quantity_sold, on='shop_id')

In [18]:
store_info_data["shop_profile"] = store_info_data["shop_profile"].replace({"High": 2, "Moderate": 1, "Low": 0})

In [19]:
print(store_info_data.head())

   shop_id  shop_area_sq_ft  shop_profile  unique_customers  transactions  \
0  SHOP047              528           1.0               928          1302   
1  SHOP009              676           2.0              2498          3493   
2  SHOP083              676           0.0              1900          2718   
3  SHOP117              676           0.0              2037          3158   
4  SHOP042              676           0.0              1841          2495   

   unique_items  items  revenue  max_item_price  min_item_price  \
0            19   3288   842960            2200            2200   
1            19   8411  1970870            2610            2610   
2            20   6644  1691985            3080            3080   
3            19   8073  2325980            1600            1600   
4            19   5844  1340215            2640            2640   

   max_quantity_sold  min_quantity_sold  
0                 22                 22  
1                 28                 28  
2       

In [20]:
non_empty_profiles = store_info_data[~store_info_data['shop_profile'].isna()]
empty_profiles = store_info_data[store_info_data['shop_profile'].isna()]

# print(non_empty_profiles)
# print(empty_profiles)

In [21]:
X = non_empty_profiles.drop(columns=["shop_id", "shop_profile"])
y = non_empty_profiles["shop_profile"]

In [29]:

from sklearn.metrics import f1_score
# Get the scores from various models
def get_score (model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    return  f1_score(y_test,y_pred_test, average = 'micro')

def get_score_train (model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    return  f1_score(y_train,y_pred_train, average = 'micro'), f1_score(y_test,y_pred_test, average = 'micro')

In [52]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

scores_l = [0]
scores_knn = [0]
scores_svc = [0]
scores_rfc = [0]
scores_gnb = [0]

splits=10

kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)

from sklearn.preprocessing import StandardScaler,RobustScaler

def sub_lists (l):
    lists = [[]]
    for i in range(len(l) + 1):
        for j in range(i):
            lists.append(l[j: i])
    return lists
 
l1 = X.columns.to_list()
combinations = sub_lists(l1)
scores = []

for combination in combinations:
  if(len(combination)>0):
      s1=0
      s2=0
      s3=0
      for train_idx, test_idx in kf.split(X,y):
          X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
          s1 += get_score(LogisticRegression(max_iter = 1000), X_train, X_test, y_train, y_test)
          
          s2 += get_score(SVC(), X_train, X_test, y_train, y_test)
          
          s3 += get_score(GaussianNB(), X_train, X_test, y_train, y_test)
          
      scores_l.append(s1/splits)
      scores_svc.append(s2/splits)
      scores_gnb.append(s3/splits)
      if(s1/splits > 0.66):
        print("s1", combination, s1/splits)
      if(s2/splits > 0.66):
        print("s2", combination, s2/splits)
      if(s3/splits > 0.66):
        print("s3", combination, s3/splits)
      # for n in range(27,29):
      
      #   scores = 0
      #   for train_idx, test_idx in kf.split(X,y):
          
      #     X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
      #     scores += (get_score(RandomForestClassifier(n), X_train, X_test, y_train, y_test))

      #     # scores += get_score(KNeighborsClassifier(n_neighbors=n), X_train, X_test, y_train, y_test)
      #     # if(get_score(KNeighborsClassifier(n_neighbors=n), X_train, X_test, y_train, y_test) == 0.76):
      #     #     print(combination, n)
      #   scores_knn.append(scores/10)
      #   if(round(scores*100) >= 66*10):
      #     print(n, combination)
      
      

print("Logistic Regression scores are: ", max(scores_l), "\n")
print("K Nearest Neighbours scores are: ", max(scores_knn), "\n")
print("Support Vector Machines scores are: ",max(scores_svc), "\n")
print("Random Forest Classifier scores are: ",max(scores_rfc), "\n")
print("Gaussian N B scores are: ",max(scores_gnb), "\n")

s1 ['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items'] 0.67
Logistic Regression scores are:  0.67 

K Nearest Neighbours scores are:  0 

Support Vector Machines scores are:  0.5700000000000001 

Random Forest Classifier scores are:  0 

Gaussian N B scores are:  0.6399999999999999 



In [23]:
final_scores = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for combination in combinations:
  
  if(len(combination)>3):
    for n in [28]:
      scores = 0
      for train_idx, test_idx in kf.split(X,y):
     
      
        X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
        scores += (get_score(RandomForestClassifier(n), X_train, X_test, y_train, y_test))

        final_scores.append(scores/splits)
        if(round(scores*100) >= 63*splits):
          print(n, combination)
print(max(final_scores))


0.59


In [24]:
final_scores = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for combination in combinations:
  
  if(len(combination)>3): 
    for n in range(2,50):
      scores = 0
      for train_idx, test_idx in kf.split(X,y):
        
        X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
        scores += (get_score(KNeighborsClassifier(n_neighbors=n), X_train, X_test, y_train, y_test))

        final_scores.append(scores/splits)
        if(round(scores*100) >= 66*splits):
          print(n, combination, scores/splits)
print(max(final_scores))

0.5900000000000001


In [25]:
final_scores = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for combination in combinations:
  if(len(combination)>3): 
    for n in range(1):
      scores = 0
      for train_idx, test_idx in kf.split(X,y):
        
        X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
        scores += (get_score(GaussianNB(), X_train, X_test, y_train, y_test))

        final_scores.append(scores/splits)
        if(round(scores*100) >= 61*splits):
          print(n, combination, scores/splits)
print(max(final_scores))

0 ['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items', 'revenue', 'max_item_price', 'min_item_price', 'max_quantity_sold', 'min_quantity_sold'] 0.61
0.61


In [26]:
final_scores = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for combination in combinations:
  
  if(len(combination)>3): 
    for n in range(1):
      scores = 0
      for train_idx, test_idx in kf.split(X,y):
        
        X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
        scores += (get_score(SVC(), X_train, X_test, y_train, y_test))

        final_scores.append(scores/splits)
        if(round(scores*100) >= 61*splits):
          print(n, combination, scores/splits)
print(max(final_scores))

0.53


In [27]:
! pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import xgboost as xgb
final_scores = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for combination in combinations:
  
  if(len(combination)>3): 
    for n in range(100, 300, 100):
      scores = 0
      for train_idx, test_idx in kf.split(X,y):
        
        X_train, X_test, y_train, y_test = X[X.columns.intersection(combination)].iloc[train_idx], X[X.columns.intersection(combination)].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
        scores += (get_score(xgb.XGBClassifier(n_estimators=n, objective='multi:softprob', tree_method='hist', eta=0.1, max_depth=5), X_train, X_test, y_train, y_test))

        final_scores.append(scores/splits)
        if(round(scores*100) >= 61*splits):
          print(n, combination, scores/splits)
print(max(final_scores))

200 ['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items'] 0.62
0.62


In [47]:
score_arr = []
splits = 5
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for train_idx, test_idx in kf.split(X,y):
  X_train, X_test, y_train, y_test = X[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']].iloc[train_idx], X[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
  score_arr.append(get_score(xgb.XGBClassifier(n_estimators=200, objective='multi:softprob', tree_method='hist', eta=0.1, max_depth=5), X_train, X_test, y_train, y_test))
print(score_arr)
print(sum(score_arr)/splits)

[0.75, 0.7, 0.5, 0.55, 0.6]
0.62


In [61]:
score_arr = []
splits = 3
kf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=10)
for train_idx, test_idx in kf.split(X,y):
  X_train, X_test, y_train, y_test = X[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']].iloc[train_idx], X[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']].iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
  score_arr.append(get_score_train(LogisticRegression(max_iter = 1000), X_train, X_test, y_train, y_test))
print(score_arr)
print("test score:", sum([x[1] for x in score_arr])/splits)
print("train_score:", sum([x[0] for x in score_arr])/splits)

[(0.7424242424242424, 0.5882352941176471), (0.7014925373134329, 0.696969696969697), (0.6567164179104478, 0.7272727272727273)]
test score: 0.6708259061200238
train_score: 0.7002110658827077


In [67]:
lr = LogisticRegression(max_iter = 1000)
lr.fit(X[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']],y)

X_test = empty_profiles.drop(columns=["shop_id", "shop_profile"])

y_test = lr.predict(X_test[['shop_area_sq_ft', 'unique_customers', 'transactions', 'unique_items', 'items']])

test_preds = pd.DataFrame({'shop_id':empty_profiles['shop_id'],'shop_profile':y_test})

In [63]:
len(test_preds)

24

In [70]:
test_preds["shop_profile"] = test_preds["shop_profile"].replace({2: "High", 1:"Moderate", 0:"Low"})

In [71]:
filename = 'DataStorm087_Day1_2.csv'
test_preds.to_csv(filename,index=False)